# Part 1: Setup + Document Loading

**Goal**: Load DataFlow's enterprise documents for RAG system


## Requirements
```
langchain==0.1.0
langchain-community==0.0.13
faiss-cpu==1.7.4
sentence-transformers==2.2.2
pandas==2.0.3
```

In [37]:
# Imports and setup
from pathlib import Path
from typing import List
from langchain.document_loaders import TextLoader, CSVLoader, JSONLoader, UnstructuredMarkdownLoader
from langchain.schema import Document

# Path to enterprise documents
KNOWLEDGE_BASE_PATH = Path("./enterprise_knowledge_base")

print("✅ Setup complete!")

Setup complete!


## Load All Documents

In [38]:
def load_enterprise_documents(base_path: Path) -> List[Document]:
    """Load all documents recursively with proper metadata"""
    
    all_docs = []
    
    print(" Loading DataFlow's documents...")
    
    # Process each department folder
    for dept_path in base_path.iterdir():
        if not dept_path.is_dir():
            continue
            
        department = dept_path.name
        print(f" {department}...")
        
        # Get ALL files recursively
        files = [f for f in dept_path.rglob("*") if f.is_file()]
        
        for file_path in files:
            try:
                # Choose loader by extension
                ext = file_path.suffix.lower()
                if ext == '.csv':
                    loader = CSVLoader(str(file_path))
                elif ext == '.json':
                    loader = JSONLoader(str(file_path), jq_schema='.', text_content=False)
                elif ext == '.md':
                    loader = UnstructuredMarkdownLoader(str(file_path))
                else:
                    loader = TextLoader(str(file_path), encoding='utf-8')
                
                # Load and add metadata
                docs = loader.load()
                for doc in docs:
                    doc.metadata.update({
                        "department": department,
                        "source_file": file_path.name,
                        "file_type": ext
                    })
                
                all_docs.extend(docs)
                rel_path = file_path.relative_to(dept_path)
                print(f"    {rel_path}")
                
            except Exception as e:
                print(f"   {file_path.name}: {str(e)[:30]}...")
    
    # Quick summary
    departments = set(doc.metadata['department'] for doc in all_docs)
    total_chars = sum(len(doc.page_content) for doc in all_docs)
    
    print(f"\n LOADED: {len(all_docs)} documents from {len(departments)} departments")
    print(f"Content: {total_chars:,} characters")
    print(f" Departments: {', '.join(sorted(departments))}")
    
    return all_docs

# Load all documents
documents = load_enterprise_documents(KNOWLEDGE_BASE_PATH)

 Loading DataFlow's documents...
 business_data...
   billing_and_pricing.csv: Error loading enterprise_knowl...
   customer_analytics.csv: Error loading enterprise_knowl...
    integration_partners.csv
 customer_facing...
   api_documentation.json: jq package not found, please i...
    competitive_analysis.txt
    product_user_guide.markdown
    terms_of_service.markdown
    troubleshooting_guide.txt
 internal_operations...
    hr_policies/employee_handbook.txt
   onboarding_checklist.json: jq package not found, please i...
   release_notes.json: jq package not found, please i...
   sales_playbook.json: jq package not found, please i...
    support_operations/customer_support_procedures.markdown
    support_operations/system_architecture.markdown
 legal_compliance...
    compliance_certifications.csv
    privacy_policy.txt
    security_policies.txt
    terms_of_service.markdown

 LOADED: 85 documents from 4 departments
Content: 167,546 characters
 Departments: business_data, customer_

## Validation

In [39]:
# Quick validation
print("🔍 Validation:")
print(f"   Documents: {len(documents)} (target: 20+)")
print(f"   Departments: {len(set(doc.metadata['department'] for doc in documents))} (target: 4)")
print(f"   Content: {sum(len(doc.page_content) for doc in documents):,} chars (target: 10,000+)")

if len(documents) >= 15:
    print("\n SUCCESS! Ready for Part 2: Text Chunking")
else:
    print("\n Low document count - check folder structure")

🔍 Validation:
   Documents: 85 (target: 20+)
   Departments: 4 (target: 4)
   Content: 167,546 chars (target: 10,000+)

 SUCCESS! Ready for Part 2: Text Chunking


# Part 2: Text Chunking

**Goal**: Transform 212 documents into optimally-sized chunks for RAG
**Why Critical**: Bad chunking = bad RAG responses. Good chunking = accurate answers.


In [40]:
# Imports for text chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List
from langchain.schema import Document

print("Text chunking tools imported!")
print(f" Starting with {len(documents)} documents from Part 1")

Text chunking tools imported!
 Starting with 85 documents from Part 1


## Smart Chunking Strategy

**Industry Best Practice**: 1000 characters with 200 overlap
- **Why 1000 chars?** Perfect balance for embedding models
- **Why 200 overlap?** Preserves context across chunks
- **Recursive splitting**: Tries sentences, then words, then characters

In [41]:
def create_smart_chunks(documents: List[Document]) -> List[Document]:
    """Split documents into optimal chunks for RAG"""
    
    print(" Creating smart chunks...")
    
    # Industry-standard chunking settings
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,        # Optimal for embedding models
        chunk_overlap=200,      # Preserve context
        length_function=len,    # Character-based
        separators=[            # Try these in order:
            "\n\n",              # Paragraphs first
            "\n",                # Then lines
            ". ",                # Then sentences
            " ",                 # Then words
            "",                  # Finally characters
        ]
    )
    
    all_chunks = []
    stats = {
        "original_docs": len(documents),
        "total_chunks": 0,
        "by_department": {},
        "by_file_type": {}
    }
    
    # Process each department
    for dept in set(doc.metadata['department'] for doc in documents):
        dept_docs = [doc for doc in documents if doc.metadata['department'] == dept]
        dept_chunks = []
        
        print(f" {dept}: {len(dept_docs)} docs → ", end="")
        
        for doc in dept_docs:
            # Split the document
            chunks = text_splitter.split_documents([doc])
            
            # Add chunk metadata
            for i, chunk in enumerate(chunks):
                chunk.metadata.update({
                    "chunk_id": f"{doc.metadata['source_file']}_{i}",
                    "chunk_index": i,
                    "total_chunks": len(chunks),
                    "chunk_size": len(chunk.page_content)
                })
            
            dept_chunks.extend(chunks)
            
            # Track stats
            file_type = doc.metadata.get('file_type', 'unknown')
            stats["by_file_type"][file_type] = stats["by_file_type"].get(file_type, 0) + len(chunks)
        
        stats["by_department"][dept] = len(dept_chunks)
        stats["total_chunks"] += len(dept_chunks)
        all_chunks.extend(dept_chunks)
        
        print(f"{len(dept_chunks)} chunks")
    
    print(f"\n CHUNKING COMPLETE:")
    print(f"    Original: {stats['original_docs']} documents")
    print(f"    Created: {stats['total_chunks']} chunks")
    print(f"    Ratio: {stats['total_chunks'] / stats['original_docs']:.1f} chunks per document")
    
    return all_chunks

# Create chunks
chunks = create_smart_chunks(documents)

 Creating smart chunks...
 internal_operations: 3 docs → 57 chunks
 legal_compliance: 28 docs → 80 chunks
 customer_facing: 4 docs → 82 chunks
 business_data: 50 docs → 50 chunks

 CHUNKING COMPLETE:
    Original: 85 documents
    Created: 269 chunks
    Ratio: 3.2 chunks per document


## Analyze Chunk Quality

In [42]:
def analyze_chunk_quality(chunks: List[Document]):
    """Analyze chunk distribution and quality"""
    
    print(" CHUNK QUALITY ANALYSIS")
    print("-" * 30)
    
    # Size analysis
    sizes = [len(chunk.page_content) for chunk in chunks]
    avg_size = sum(sizes) / len(sizes)
    min_size = min(sizes)
    max_size = max(sizes)
    
    print(f"Size Distribution:")
    print(f"   Average: {avg_size:.0f} characters")
    print(f"   Range: {min_size} - {max_size} characters")
    
    # Size buckets
    buckets = {
        "Small (0-500)": sum(1 for s in sizes if s <= 500),
        "Medium (500-1000)": sum(1 for s in sizes if 500 < s <= 1000),
        "Large (1000+)": sum(1 for s in sizes if s > 1000)
    }
    
    print(f"\n Size Distribution:")
    for bucket, count in buckets.items():
        percentage = (count / len(chunks)) * 100
        print(f"   {bucket}: {count} chunks ({percentage:.1f}%)")
    
    # Department distribution
    by_dept = {}
    for chunk in chunks:
        dept = chunk.metadata['department']
        by_dept[dept] = by_dept.get(dept, 0) + 1
    
    print(f"\n By Department:")
    for dept, count in sorted(by_dept.items()):
        percentage = (count / len(chunks)) * 100
        print(f"   {dept}: {count} chunks ({percentage:.1f}%)")
    
    # Quality assessment
    optimal_chunks = sum(1 for s in sizes if 500 <= s <= 1000)
    quality_score = (optimal_chunks / len(chunks)) * 100
    
    print(f"\n Quality Score: {quality_score:.1f}%")
    print(f"   ({optimal_chunks}/{len(chunks)} chunks in optimal range)")
    
    if quality_score >= 70:
        print(" Excellent chunking quality!")
    elif quality_score >= 50:
        print(" Good chunking quality")
    else:
        print("️ Consider adjusting chunk size")

analyze_chunk_quality(chunks)

 CHUNK QUALITY ANALYSIS
------------------------------
Size Distribution:
   Average: 661 characters
   Range: 3 - 998 characters

 Size Distribution:
   Small (0-500): 86 chunks (32.0%)
   Medium (500-1000): 183 chunks (68.0%)
   Large (1000+): 0 chunks (0.0%)

 By Department:
   business_data: 50 chunks (18.6%)
   customer_facing: 82 chunks (30.5%)
   internal_operations: 57 chunks (21.2%)
   legal_compliance: 80 chunks (29.7%)

 Quality Score: 68.0%
   (183/269 chunks in optimal range)
 Good chunking quality


## Sample Chunks Review

In [43]:
# Show sample chunks from different departments
print(" SAMPLE CHUNKS REVIEW")
print("-" * 25)

departments = list(set(chunk.metadata['department'] for chunk in chunks))

for dept in departments[:3]:  # Show first 3 departments
    dept_chunks = [c for c in chunks if c.metadata['department'] == dept]
    if dept_chunks:
        sample = dept_chunks[0]  # First chunk from department
        
        print(f"\n{dept.upper()}:")
        print(f"   File: {sample.metadata['source_file']}")
        print(f"   Size: {len(sample.page_content)} chars")
        print(f"   Preview: {sample.page_content[:150]}...")
        print(f"   Metadata: {sample.metadata['chunk_id']}")

print(f"\n Total chunks ready for vector storage: {len(chunks)}")

 SAMPLE CHUNKS REVIEW
-------------------------

BUSINESS_DATA:
   File: integration_partners.csv
   Size: 303 chars
   Preview: Partner_Name: Salesforce
Integration_Type: CRM
Setup_Difficulty: 3
Support_Level: Full
Documentation_Link: https://docs.dataflow.com/salesforce
Certif...
   Metadata: integration_partners.csv_0

LEGAL_COMPLIANCE:
   File: compliance_certifications.csv
   Size: 276 chars
   Preview: Certification: SOC 2 Type II
Status: Compliant
Date_Achieved: 2024-01-15
Renewal_Date: 2026-01-15
Audit_Frequency: Annual
Last_Audit_Result: Pass
Busi...
   Metadata: compliance_certifications.csv_0

CUSTOMER_FACING:
   File: competitive_analysis.txt
   Size: 877 chars
   Preview: DataFlow Solutions Competitive Analysis
Last Updated: June 8, 2025

This document analyzes DataFlow Solutions’ position in the SaaS BI and data visual...
   Metadata: competitive_analysis.txt_0

 Total chunks ready for vector storage: 269


## Validation

In [44]:
# Validate chunking success
print("🔍 CHUNKING VALIDATION")
print("-" * 20)

# Basic checks
total_original_chars = sum(len(doc.page_content) for doc in documents)
total_chunk_chars = sum(len(chunk.page_content) for chunk in chunks)
content_preserved = (total_chunk_chars / total_original_chars) * 100

checks = [
    (len(chunks) > len(documents), f"More chunks than docs: {len(chunks)} > {len(documents)}"),
    (content_preserved >= 90, f"Content preserved: {content_preserved:.1f}%"),
    (all('chunk_id' in c.metadata for c in chunks), "All chunks have IDs"),
    (all('department' in c.metadata for c in chunks), "All chunks have departments")
]

all_passed = True
for passed, message in checks:
    status = "pass" if passed else "fail"
    print(f"   {status} {message}")
    if not passed:
        all_passed = False

if all_passed:
    print("\nSUCCESS! Chunks ready for Part 3: Vector Embeddings")
else:
    print("\n Some validation checks failed")

🔍 CHUNKING VALIDATION
--------------------
   pass More chunks than docs: 269 > 85
   pass Content preserved: 106.2%
   pass All chunks have IDs
   pass All chunks have departments

SUCCESS! Chunks ready for Part 3: Vector Embeddings


# Part 3: Vector Embeddings & Search

**Goal**: Transform text chunks into searchable mathematical vectors
**Why Critical**: This enables semantic search - finding meaning, not just keywords


Then run: `pip install sentence-transformers==4.1.0 huggingface-hub==0.32.4 langchain-huggingface`



In [45]:
# Modern imports (no deprecation warnings)
try:
    # Modern approach - no deprecation warnings
    from langchain_huggingface import HuggingFaceEmbeddings
    print(" Using modern langchain-huggingface (recommended)")
    modern_import = True
except ImportError:
    # Fallback to deprecated version if needed
    from langchain.embeddings import HuggingFaceEmbeddings
    print(" Using deprecated import (consider upgrading)")
    print(" Run: pip install langchain-huggingface")
    modern_import = False

from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
import numpy as np
from typing import List, Tuple

print(" Vector tools imported!")
print(f"Ready to embed {len(chunks)} chunks from Part 2")

 Using modern langchain-huggingface (recommended)
 Vector tools imported!
Ready to embed 269 chunks from Part 2


## Setup Embedding Model

**Model Choice**: `all-MiniLM-L6-v2`
- **Fast**: Perfect for development and production
- **Accurate**: Great semantic understanding
- **Compact**: 384 dimensions (vs 1536 for OpenAI)
- **Free**: No API costs

In [46]:
def setup_embedding_model():
    """Initialize the embedding model for vector creation"""
    
    print("Loading embedding model...")
    
    # Use production-grade embedding model
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    
    # Modern LangChain wrapper (no deprecation warnings)
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={'device': 'cpu'},  # Use CPU for compatibility
        encode_kwargs={'normalize_embeddings': True}  # Better for similarity search
    )
    
    print(f"Model loaded: {model_name}")
    print(f"Vector dimensions: 384")
    print(f"Device: CPU (production compatible)")
    
    if modern_import:
        print(" Using modern non-deprecated embeddings!")
    
    return embeddings

# Setup embeddings
embeddings = setup_embedding_model()

Loading embedding model...
Model loaded: sentence-transformers/all-MiniLM-L6-v2
Vector dimensions: 384
Device: CPU (production compatible)
 Using modern non-deprecated embeddings!


## Create Vector Store

**FAISS**: Facebook's vector search library
- Powers Instagram recommendations
- Billion-scale vector search
- Lightning-fast similarity search
- Industry standard for RAG systems

In [47]:
def create_vector_store(chunks: List, embeddings) -> FAISS:
    """Create FAISS vector store from text chunks"""
    
    print(" Creating vector embeddings...")
    print(" This may take 30-60 seconds...")
    
    # Create vector store with FAISS
    vector_store = FAISS.from_documents(
        documents=chunks,
        embedding=embeddings
    )
    
    print(f" Vector store created!")
    print(f" Vectors: {len(chunks)}")
    print(f" Dimensions: 384 per vector")
    print(f"Total size: ~{len(chunks) * 384 * 4 / 1024 / 1024:.1f} MB")
    
    return vector_store

# Create the vector store
vector_store = create_vector_store(chunks, embeddings)

 Creating vector embeddings...
 This may take 30-60 seconds...
 Vector store created!
 Vectors: 269
 Dimensions: 384 per vector
Total size: ~0.4 MB


## Test Semantic Search

In [48]:
def test_semantic_search(vector_store: FAISS, test_queries: List[str]):
    """Test the vector store with realistic customer queries"""
    
    print(" TESTING SEMANTIC SEARCH")
    print("-" * 30)
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n Query {i}: '{query}'")
        
        # Search for most relevant chunks
        results = vector_store.similarity_search(
            query=query, 
            k=3  # Top 3 most relevant chunks
        )
        
        print(f" Found {len(results)} relevant chunks:")
        
        for j, result in enumerate(results, 1):
            dept = result.metadata['department']
            file = result.metadata['source_file']
            preview = result.page_content[:100].replace('\n', ' ')
            
            print(f"   {j}.  {dept} |  {file}")
            print(f"      Preview: {preview}...")

# Test with realistic customer queries
test_queries = [
    "What are your pricing plans?",
    "How do I integrate with your API?",
    "What is your privacy policy?",
    "I'm having trouble with authentication",
    "Employee handbook and HR policies"
]

test_semantic_search(vector_store, test_queries)

 TESTING SEMANTIC SEARCH
------------------------------

 Query 1: 'What are your pricing plans?'
 Found 3 relevant chunks:
   1.  customer_facing |  competitive_analysis.txt
      Preview: **Pricing**: - Pro: $10/user/month, Premium: $20/user/month, Premium Capacity: $4,995/month. - DataF...
   2.  legal_compliance |  terms_of_service.markdown
      Preview: 3.2 Payment Terms - **Methods**: Credit card, invoice, ACH for Enterprise (billing_and_pricing.csv)....
   3.  customer_facing |  terms_of_service.markdown
      Preview: 3.2 Payment Terms - **Methods**: Credit card, invoice, ACH for Enterprise (billing_and_pricing.csv)....

 Query 2: 'How do I integrate with your API?'
 Found 3 relevant chunks:
   1.  customer_facing |  product_user_guide.markdown
      Preview: ### 10.4 References - API Documentation: `api_documentation.json` - Troubleshooting Guide: `troubles...
   2.  internal_operations |  system_architecture.markdown
      Preview: ---  ## 3. API Gateway and Microservices Th

## Search Quality Analysis

In [49]:
def analyze_search_quality(vector_store: FAISS):
    """Analyze the quality and coverage of semantic search"""
    
    print(" SEARCH QUALITY ANALYSIS")
    print("-" * 30)
    
    # Test queries for each department
    dept_queries = {
        "business_data": "pricing and billing information",
        "customer_facing": "product features and user guide",
        "internal_operations": "employee policies and procedures", 
        "legal_compliance": "privacy and terms of service"
    }
    
    coverage_score = 0
    total_tests = len(dept_queries)
    
    for dept, query in dept_queries.items():
        print(f"\n Testing {dept} coverage...")
        
        results = vector_store.similarity_search(query, k=5)
        
        # Check if top results are from the right department
        dept_matches = sum(1 for r in results if r.metadata['department'] == dept)
        accuracy = (dept_matches / len(results)) * 100 if results else 0
        
        print(f"   Query: '{query}'")
        print(f"   Accuracy: {dept_matches}/{len(results)} = {accuracy:.1f}%")
        
        if accuracy >= 60:  # At least 3/5 results from correct dept
            coverage_score += 1
            print(f"   Good coverage")
        else:
            print(f"    Needs improvement")
    
    overall_score = (coverage_score / total_tests) * 100
    
    print(f"\n OVERALL SEARCH QUALITY: {overall_score:.1f}%")
    print(f"   ({coverage_score}/{total_tests} departments with good coverage)")
    
    if overall_score >= 75:
        print(" Excellent search quality!")
    elif overall_score >= 50:
        print(" Good search quality")
    else:
        print(" Consider more diverse chunks or better embeddings")
    
    return overall_score

quality_score = analyze_search_quality(vector_store)

 SEARCH QUALITY ANALYSIS
------------------------------

 Testing business_data coverage...
   Query: 'pricing and billing information'
   Accuracy: 0/5 = 0.0%
    Needs improvement

 Testing customer_facing coverage...
   Query: 'product features and user guide'
   Accuracy: 2/5 = 40.0%
    Needs improvement

 Testing internal_operations coverage...
   Query: 'employee policies and procedures'
   Accuracy: 2/5 = 40.0%
    Needs improvement

 Testing legal_compliance coverage...
   Query: 'privacy and terms of service'
   Accuracy: 4/5 = 80.0%
   Good coverage

 OVERALL SEARCH QUALITY: 25.0%
   (1/4 departments with good coverage)
 Consider more diverse chunks or better embeddings


## Save Vector Store

In [50]:
# Save vector store for production use
def save_vector_store(vector_store: FAISS, save_path: str = "dataflow_vector_store"):
    """Save vector store to disk for reuse"""
    
    print(f"💾 Saving vector store to '{save_path}'...")
    
    try:
        vector_store.save_local(save_path)
        print(f"Vector store saved successfully!")
        print(f"Location: {save_path}/")
        print(f" Can be loaded later with: FAISS.load_local('{save_path}', embeddings)")
        return True
    except Exception as e:
        print(f"❌ Save failed: {e}")
        return False

# Save the vector store
save_success = save_vector_store(vector_store)

💾 Saving vector store to 'dataflow_vector_store'...
Vector store saved successfully!
Location: dataflow_vector_store/
 Can be loaded later with: FAISS.load_local('dataflow_vector_store', embeddings)


## Validation

In [51]:
# Final validation
print(" VECTOR STORE VALIDATION")
print("-" * 25)

# Test basic functionality
test_query = "pricing information"
test_results = vector_store.similarity_search(test_query, k=1)

checks = [
    (len(test_results) > 0, "Vector search returns results"),
    (hasattr(vector_store, 'index'), "FAISS index created"),
    (save_success, "Vector store saved successfully"),
    (len(chunks) > 0, f"All {len(chunks)} chunks embedded"),
    (modern_import, "Using modern non-deprecated imports")
]

all_passed = True
for passed, message in checks:
    status = "pass" if passed else "fail"
    print(f"   {status} {message}")
    if not passed:
        all_passed = False

if all_passed:
    print("\n SUCCESS! Vector store ready for Part 4: RAG Agent")
    print(" You now have a production-grade semantic search system!")
    print(f" Search quality: {quality_score:.1f}% accuracy")
else:
    print("\n️ Some validation checks failed")

 VECTOR STORE VALIDATION
-------------------------
   pass Vector search returns results
   pass FAISS index created
   pass Vector store saved successfully
   pass All 269 chunks embedded
   pass Using modern non-deprecated imports

 SUCCESS! Vector store ready for Part 4: RAG Agent
 You now have a production-grade semantic search system!
 Search quality: 25.0% accuracy


# Part 4: RAG Agent - Complete Intelligent System

**Goal**: Connect vector search to LLM for intelligent customer service



## LLM Setup Required
**Ollama (Free, Local)**
```bash
# Install Ollama from https://ollama.ai
ollama pull llama3.2  # Download model
ollama serve         # Start server
```

**Alternative: OpenAI (if you prefer)**
```bash
pip install openai
# Set OPENAI_API_KEY environment variable
```

In [29]:
# Imports for RAG agent
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage
import time
from typing import Dict, List, Any

# Simple Ollama setup
llm = None

try:
    from langchain.llms import Ollama
    llm = Ollama(model="llama3.2", base_url="http://localhost:11434")
    # Test connection
    test_response = llm.invoke("Hello")
    print(" Ollama LLM connected successfully!")
    print("Using free local LLM")
    print(f"Vector store ready: {len(chunks)} chunks")
except Exception as e:
    print(f" Ollama connection failed: {e}")
    print(" Make sure Ollama is running: ollama serve")
    print(" And model is downloaded: ollama pull llama3.2")
    llm = None

## Create Customer Service Prompt

In [65]:
# Professional customer service prompt
CUSTOMER_SERVICE_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template="""You are DataFlow's helpful customer service assistant. Your job is to provide accurate, friendly, and professional support to customers.

INSTRUCTIONS:
- Use the provided context to answer questions accurately
- Be concise but thorough in your explanations
- If information isn't in the context, say "I don't have that specific information" and suggest contacting support
- Always maintain a helpful and professional tone
- For technical questions, provide step-by-step guidance when possible

CONTEXT:
{context}

CUSTOMER QUESTION:
{question}

RESPONSE:"""
)

print("✅ Professional customer service prompt created")
print("🎯 Optimized for helpful, accurate responses")

Professional customer service prompt created
 Optimized for helpful, accurate responses


## Build RAG Chain

In [67]:
def create_rag_chain(vector_store, llm, prompt_template):
    """Create production RAG chain"""

    if not llm:
        print(" No LLM available - cannot create RAG chain")
        print(" Please install Ollama or set up OpenAI API key")
        return None

    print("🔗 Creating RAG chain...")

    # Create retrieval QA chain
    rag_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  # Stuff all context into prompt
        retriever=vector_store.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 4}  # Retrieve top 4 most relevant chunks
        ),
        chain_type_kwargs={
            "prompt": prompt_template
        },
        return_source_documents=True  # Show which documents were used
    )

    print(" RAG chain created successfully!")
    print(" Retriever: Top 4 most relevant chunks")
    print(" LLM: Ready for customer questions")
    print(" Source attribution: Enabled")

    return rag_chain

# Create the RAG chain
rag_chain = create_rag_chain(vector_store, llm, CUSTOMER_SERVICE_PROMPT)

## Customer Service Agent

In [68]:
class DataFlowCustomerAgent:
    """Professional customer service agent with simple conversation tracking"""

    def __init__(self, rag_chain):
        self.rag_chain = rag_chain
        # Simple conversation tracking (no deprecated memory)
        self.conversation_history = []
        self.conversation_count = 0
        self.response_times = []

        print("🤖 DataFlow Customer Service Agent initialized")

    def ask(self, question: str) -> Dict[str, Any]:
        """Ask the agent a question and get a comprehensive response"""

        if not self.rag_chain:
            return {
                "answer": "I'm sorry, but I'm not properly configured right now. Please contact our support team directly.",
                "sources": [],
                "response_time": 0,
                "error": "No LLM available"
            }

        start_time = time.time()

        try:
            # Get response from RAG chain
            response = self.rag_chain.invoke({"query": question})

            end_time = time.time()
            response_time = end_time - start_time

            # Simple conversation tracking
            self.conversation_history.append({
                "question": question,
                "answer": response["result"],
                "timestamp": start_time
            })

            # Track metrics
            self.conversation_count += 1
            self.response_times.append(response_time)

            # Extract source information
            sources = []
            if "source_documents" in response:
                for doc in response["source_documents"]:
                    sources.append({
                        "department": doc.metadata.get("department", "unknown"),
                        "file": doc.metadata.get("source_file", "unknown"),
                        "preview": doc.page_content[:100] + "..."
                    })

            return {
                "answer": response["result"],
                "sources": sources,
                "response_time": response_time,
                "conversation_turn": self.conversation_count
            }

        except Exception as e:
            return {
                "answer": f"I apologize, but I encountered an error. Please try rephrasing or contact support.",
                "sources": [],
                "response_time": time.time() - start_time,
                "error": str(e)
            }

    def get_stats(self) -> Dict[str, Any]:
        """Get agent performance statistics"""

        if not self.response_times:
            return {"conversations": 0, "avg_response_time": 0}

        return {
            "conversations": self.conversation_count,
            "avg_response_time": sum(self.response_times) / len(self.response_times),
            "fastest_response": min(self.response_times),
            "slowest_response": max(self.response_times),
            "total_history": len(self.conversation_history)
        }

    def get_conversation_history(self, last_n: int = 5):
        """Get recent conversation history"""
        return self.conversation_history[-last_n:] if self.conversation_history else []

# Create the customer service agent
agent = DataFlowCustomerAgent(rag_chain)
print("✅ Customer service agent ready!")

## Test Customer Scenarios

In [73]:
def test_customer_scenarios(agent):
    """Test agent with realistic customer service scenarios"""

    print(" TESTING CUSTOMER SERVICE SCENARIOS")
    print("=" * 45)

    # Realistic customer questions
    scenarios = [
        {
            "question": "What are your pricing plans and how much does the premium plan cost?",
            "category": "Billing",
            "expected_dept": "business_data"
        },
        {
            "question": "How do I authenticate with your API? I'm getting authentication errors.",
            "category": "Technical Support",
            "expected_dept": "customer_facing"
        },
        {
            "question": "What data do you collect and how do you protect my privacy?",
            "category": "Privacy/Legal",
            "expected_dept": "legal_compliance"
        }
    ]

    results = []

    for i, scenario in enumerate(scenarios, 1):
        print(f"\n📞 Scenario {i}: {scenario['category']}")
        print(f"❓ Question: {scenario['question']}")
        print("-" * 50)

        # Get agent response
        response = agent.ask(scenario["question"])

        print(f" Agent Response:")
        print(f"   {response['answer']}")  # Show complete response

        print(f"\n Sources Used:")
        for j, source in enumerate(response['sources'][:3], 1):  # Show top 3 sources
            print(f"   {j}.  {source['department']} - {source['file']}")

        print(f"\n Response Time: {response['response_time']:.2f} seconds")

        # Check if correct department was used
        dept_match = any(source['department'] == scenario['expected_dept'] for source in response['sources'])
        accuracy = "✅ Accurate" if dept_match else " Needs Review"
        print(f" Department Accuracy: {accuracy}")

        results.append({
            "scenario": scenario,
            "response": response,
            "accurate": dept_match
        })

    return results

# Test the scenarios
test_results = test_customer_scenarios(agent)


🎭 RUNNING CUSTOMER SCENARIOS

📞 Scenario 1: Billing
❓ Question: What are your pricing plans and how much does the premium plan cost?
--------------------------------------------------
🤖 Agent Response:
   I'm sorry, but I'm not properly configured right now. Please contact our support team directly.

📚 Sources Used:
   ⚠️ No sources provided

⏱️ Response Time: 0.00 seconds
🎯 Department Accuracy: ⚠️ Needs Review

📞 Scenario 2: Technical Support
❓ Question: How do I authenticate with your API? I'm getting authentication errors.
--------------------------------------------------
🤖 Agent Response:
   I'm sorry, but I'm not properly configured right now. Please contact our support team directly.

📚 Sources Used:
   ⚠️ No sources provided

⏱️ Response Time: 0.00 seconds
🎯 Department Accuracy: ⚠️ Needs Review

📞 Scenario 3: Privacy/Legal
❓ Question: What data do you collect and how do you protect my privacy?
--------------------------------------------------
🤖 Agent Response:
   I'm sorry, b

## Business Impact Analysis

In [74]:
def calculate_business_impact(agent, test_results):
    """Calculate measurable business impact and ROI"""

    print("BUSINESS IMPACT ANALYSIS")
    print("=" * 30)

    # Get agent performance stats
    stats = agent.get_stats()

    # Calculate accuracy
    accurate_responses = sum(1 for result in test_results if result['accurate'])
    accuracy_rate = (accurate_responses / len(test_results)) * 100 if test_results else 0

    # Business metrics
    metrics = {
        "daily_customer_questions": 50,
        "avg_human_response_time": 300,  # 5 minutes
        "hourly_support_cost": 25,
        "working_days_per_year": 250,
        "ai_accuracy_rate": accuracy_rate,
        "ai_avg_response_time": stats.get('avg_response_time', 0)
    }

    # Calculate savings
    daily_human_hours = (metrics['daily_customer_questions'] * metrics['avg_human_response_time']) / 3600
    daily_ai_hours = (metrics['daily_customer_questions'] * metrics['ai_avg_response_time']) / 3600

    hours_saved_daily = daily_human_hours - daily_ai_hours
    daily_cost_savings = hours_saved_daily * metrics['hourly_support_cost']
    annual_savings = daily_cost_savings * metrics['working_days_per_year']

    print(f" PERFORMANCE METRICS:")
    print(f"   Accuracy Rate: {accuracy_rate:.1f}%")
    print(f"   Avg Response Time: {metrics['ai_avg_response_time']:.2f} seconds")
    print(f"   Questions Handled: {stats.get('conversations', 0)}")

    print(f"\n COST ANALYSIS:")
    print(f"   Human Response Time: {metrics['avg_human_response_time']} seconds avg")
    print(f"   AI Response Time: {metrics['ai_avg_response_time']:.1f} seconds avg")

    print(f"\n BUSINESS IMPACT:")
    print(f"   Hours Saved Daily: {hours_saved_daily:.1f} hours")
    print(f"   Daily Cost Savings: ${daily_cost_savings:.2f}")
    print(f"   Annual Cost Savings: ${annual_savings:,.2f}")

    return {
        "accuracy_rate": accuracy_rate,
        "annual_savings": annual_savings,
        "hours_saved_daily": hours_saved_daily
    }

# Calculate business impact
business_impact = calculate_business_impact(agent, test_results)


💵 BUSINESS IMPACT SUMMARY
Accuracy Rate: 0.0%
Avg Response Time: 0.00 seconds
Hours Saved Daily: 4.2 hours
Daily Cost Savings: $104.17
Estimated Annual Savings: $26,041.67


## Validation

In [72]:
# Final system validation
print("🔍 FINAL SYSTEM VALIDATION")
print("=" * 30)

# System components check
components = [
    (len(chunks) > 0, f"Document chunks loaded: {len(chunks)}"),
    (vector_store is not None, "Vector store created"),
    (llm is not None, f"LLM connected: {llm_type if llm else 'None'}"),
    (rag_chain is not None, "RAG chain built"),
    (agent is not None, "Customer service agent ready")
]

all_systems_go = True
for check, message in components:
    status = "✅" if check else "❌"
    print(f"   {status} {message}")
    if not check:
        all_systems_go = False

# Performance validation
if test_results:
    accuracy = sum(1 for r in test_results if r['accurate']) / len(test_results) * 100
    print(f"\n PERFORMANCE VALIDATION:")
    print(f"   Accuracy Rate: {accuracy:.1f}%")
    print(f"   esponse Time: {agent.get_stats().get('avg_response_time', 0):.2f}s avg")
    print(f"    Business Impact: ${business_impact.get('annual_savings', 0):,.0f} annual savings")

if all_systems_go:
    print("\nSUCCESS! COMPLETE RAG SYSTEM OPERATIONAL")
    print(" DataFlow's AI customer service agent is ready for production!")

    if business_impact.get('accuracy_rate', 0) >= 75:
        print("⭐ EXCELLENT: High accuracy + strong business case")
    elif business_impact.get('accuracy_rate', 0) >= 60:
        print(" GOOD: Solid foundation for customer service automation")
    else:
        print("⚠NEEDS IMPROVEMENT: Consider fine-tuning")
else:
    print("\n⚠ PARTIAL SUCCESS: Some components need attention")
    print(" Check LLM setup (Ollama or OpenAI) for full functionality")


🔍 FINAL SYSTEM VALIDATION


NameError: name 'llm_wrapper' is not defined